     |████████████████████████████████| 100.1 MB 24 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=1e33d97005dd8af0045aca897f53ead2b704f3e14ca44a7f7a11ef3fc94d72dd
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [2]:
from PIL import Image
import face_recognition #detecting and recogniting faces
import cv2 #intracting woth image
import os # reading file name

#image = face_recognition.load_image_file("two_hero.jpg")

In [ ]:
!mkdir known

!wget https://upload.wikimedia.org/wikipedia/commons/e/ed/Elon_Musk_Royal_Society.jpg -O known/elon.jpg
#!wget C:\Users\user\Desktop -O known/bill.jpg
#!wget https://pmcvariety.files.wordpress.com/2017/05/mark-zuckerberg.jpg?w=681&h=383&crop=1 -O known/mark.jpg

--2022-05-05 06:25:12--  https://upload.wikimedia.org/wikipedia/commons/e/ed/Elon_Musk_Royal_Society.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.153.240, 2620:0:860:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.153.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239507 (234K) [image/jpeg]
Saving to: ‘known/elon.jpg’

known/elon.jpg      100%[===================>] 233.89K  --.-KB/s    in 0.09s   

2022-05-05 06:25:12 (2.55 MB/s) - ‘known/elon.jpg’ saved [239507/239507]



In [3]:
!mkdir unknown
!wget  https://upload.wikimedia.org/wikipedia/commons/a/a0/Bill_Gates_2018.jpg -O unknown/1.jpg
#!wget https://api.time.com/wp-content/uploads/2019/04/mark-zuckerberg-time-100-2019.jpg?quality=85&zoom=2 -O unknown/2.jpg

mkdir: cannot create directory ‘unknown’: File exists
--2022-05-11 02:15:41--  https://upload.wikimedia.org/wikipedia/commons/a/a0/Bill_Gates_2018.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2001:df2:e500:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 318220 (311K) [image/jpeg]
Saving to: ‘unknown/1.jpg’

unknown/1.jpg       100%[===================>] 310.76K  1.03MB/s    in 0.3s    

2022-05-11 02:15:42 (1.03 MB/s) - ‘unknown/1.jpg’ saved [318220/318220]



In [ ]:
from google.colab.patches import cv2_imshow

In [4]:
def read_img(path):
  img = cv2.imread(path)  #read image
  (h,w) = img.shape[:2] # fetch height and width
  width = 500 # hard coding width
  ratio = width / float(w)
  height = int(h*ratio)
  return cv2.resize(img,(width,height))


In [ ]:
known_encodings =[]
known_names = []
known_dir = 'known'
for file in os.listdir(known_dir):
  if '.jpg' in file:
    img = read_img(known_dir + '/' + file)
    img_enc = face_recognition.face_encodings(img)[0]

    known_encodings.append(img_enc)
    known_names.append(file.split('.')[0])
print(known_names)

['elon']


In [5]:
unknown_dir = 'unknown'
for file in os.listdir(unknown_dir):
  if '.png' in file:
    img = img_read(known_dir + '/' + file)
    img_enc = face_recognition.face_encodings(img)[0]

    results = face_recognition.compare_faces(known_encodings, img_enc)
    

# ---경계선---

In [6]:


# from google.colab.patches import cv2_imshow
import cv2
import face_recognition
import os # reading file name
import numpy as np
from sklearn.cluster import DBSCAN #clustering에 필요 
import re #문자열에서 숫자 추출

In [15]:
class Face():
  def __init__(self, frame_id, name, box, encoding):
    self.frame_id = frame_id
    self.name = name
    self.box = box
    self.encoding = encoding

class Face_clustering():
  def __init__(self):
    self.faces = []
    self.capture_dir = "captures"
    self.unique_cnt_dict = {}
    self.person_class = 0
  def capture_filename(self, frame_id):
    return "frame_{}.jpg".format(frame_id)
  def drawBox(self, frame, face):
   (top, right, bottom, left) = face.box
   #print(face.frame_id)
   cv2.rectangle(frame, (left,top), (right,bottom), (0,0,255), 2)

  def encode(self, src_file='./ironman.mp4', capture_per_sec = 1, stop=0): #frame 하나에서 얼굴 추출
    video = cv2.VideoCapture(src_file) 
    if not video.isOpened():
      print("video not opened")
      return
    
    print(video.get(cv2.CAP_PROP_FRAME_COUNT)) #총 프레임 수 
    print(video.get(cv2.CAP_PROP_FPS)) # 초당 프레임 수
    print(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    print(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_per_sec = video.get(cv2.CAP_PROP_FPS)
    total_frame = video.get(cv2.CAP_PROP_FRAME_COUNT)
   # capture_per_sec = 1
    frame_id=0
    frame_between_capture = round(frame_per_sec / capture_per_sec) #30

    if not os.path.exists(self.capture_dir): #/captures디렉토리가 없으면
      os.mkdir(self.capture_dir)

    while video.isOpened():
      ret, frame = video.read()
      frame_id+=1

      if ret is True:
        if frame_id % frame_between_capture == 0: 
          #cv2_imshow(frame)
          
          rgb_frame = frame[:,:,::-1] #BGR을 RGB 순으로 바꾸기
          face_box = face_recognition.face_locations(rgb_frame, model="hog")
          face_encodings = face_recognition.face_encodings(rgb_frame, face_box)
          # face_box.shape (top, right, bottom, left, face_개수)
          #face_encodings.shape (128 vector, face_개수)
          if not face_box:
             # print("face_box 0: %d" %frame_id)
              continue

          faces_in_frame = [] #feces_in_frame에 Face class 저장
          for box, encodings in zip(face_box, face_encodings):
            face = Face(frame_id, None, box, encodings)
            faces_in_frame.append(face)
            self.drawBox(frame, face)
          

          pathname = os.path.join(self.capture_dir, self.capture_filename(frame_id)) # captures/frame_#.jpg
          cv2.imwrite(pathname, frame) # 해당 디렉토리에 이미지 저장

          self.faces.extend(faces_in_frame) #extend는 iterable한것들을 append
          #if cv2.waitKey(1) & 0xFF == ord('q'): # waitkey 안됨
           # break
      
      else:
        break
    video.release()
    return

  def get_face_image(self, image, box):
    (top, right, bottom, left) = box
    image_height = image.shape[0]
    image_width = image.shape[1]
    box_width = right - left
    box_height = bottom - top

    top = max(top - box_height, 0)
    bottom = min(bottom + box_height, image_height - 1)
    left = max(left - box_width, 0)
    right = min(right + box_width, image_width - 1)

    return image[top:bottom, left:right]
    #image (frame_height, frame_width, rgb)

  def cluster(self):
   # print(len(self.faces))
    if len(self.faces) == 0:
      return
    encodings = [face.encoding for face in self.faces]

    cm = DBSCAN(metric="euclidean")
    cm.fit(encodings)
    # clustering 완료

    label_ids, count = np.unique(cm.labels_, return_counts=True) #0부터 labeling, -1은 분류되지 않은 encoding
    self.unique_cnt_dict = dict(zip(label_ids, count)) # 레이블당 프레임수
    print(self.unique_cnt_dict)
    self.person_class = len(label_ids) - 1 #사람 종류
    for label_id in label_ids:
      dir_name = "ID%d" % label_id
     # if label_id > -1: # -1처리는 어떻게 해야하나
      if not os.path.exists(dir_name): #/id# 디렉토리가 없으면
        os.mkdir(dir_name)
      index = np.where(cm.labels_ == label_id)[0]
      #print(index)
      for i in index:
        #  print(f'faces : {self.faces}')
        #  print(f'i : {i}')
        #  print(f'type i : {type(i)}')
          
          frame_id = self.faces[i].frame_id
          box = self.faces[i].box
          pathname = os.path.join(self.capture_dir, self.capture_filename(frame_id)) # captures/frame_#.jpg
          image = cv2.imread(pathname)
          face_image = self.get_face_image(image, box)
          filename = dir_name + '-' + self.capture_filename(frame_id) #ID1/frame_#.jpg
          pathname = os.path.join(dir_name, filename)
          cv2.imwrite(pathname, face_image)

    print('clustering done')


In [16]:
if __name__ == "__main__":
  import argparse

  #ap = argparse.ArgumentParser()
  #ap.add_argument("-e", "--encode", help="video file top encode or '0' to encode web cam")
  #ap.add_argument("-c", "--capture", default=1, type=int, help="# of frame to capture per second")
  #ap.add_argument("-s", "--stop", default=0, type=int, help="stop encoding after # seconds")
  #args = ap.parse_args()

  fc = Face_clustering()
 # if args.encode:
 #   src_file = args.encode
 #   if src_file == "0":
 #     src_file = 0
 #   fc.encode(src_file, args.capture, args.stop)
  fc.encode()


4597.0
30.0
1280.0
544.0


In [17]:
 fc.cluster()

{-1: 5, 0: 6, 1: 12, 2: 25, 3: 6}
clustering done


In [ ]:
!rm -r captures

In [ ]:
def image_to_video(self):
  for i in range(self.person_class):
    frame_id = []
    dir_name = "ID%d" % i
    for file in os.listdir(dir_name):
      if '.jpg' in file:
        id = int(re.sub(r'[^0-9]', '', file))
        frame_id.append(id)
        frame_id.sort()
  